In [ ]:
# import pandas as pd
# import nltk
# import re
# nltk.download('punkt')
# nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Projects/News/Sentiment_dataset.csv")

In [ ]:
X = df["text"]
y = df["sentiment"].values
y = y.reshape(-1,1)

In [ ]:
X = X.apply(nltk.word_tokenize)

In [ ]:
def removing_stopwords(x):
  return " ".join([i for i in x if i not in stopwords.words('english')])

In [ ]:
X = X.apply(removing_stopwords)
X

0      Billionaire investor Shark Tank star Mark Cuba...
1      Attorney Jarrett Adams recently helped overtur...
2      The Amazon Rainforest well known across world ...
3      Newark police : No officer fired single shot 2...
4      I 1970 døde det 560 mennesker den norske trafi...
                             ...                        
843    Frightening details emerged toilet block attac...
844    interview Johannesburg — 360,000 adolescents p...
845    A violent attack southern Israeli port city Ei...
846    It 's 17-year-old boy , frightened wear kippa ...
847    HONG KONG ( AFP ) - Five people killed 32 inju...
Name: text, Length: 848, dtype: object

In [ ]:
stemmer = PorterStemmer()
def stemming(x):
  return " ".join([stemmer.stem(i) for i in x.split()])

In [ ]:
X = X.apply(stemming)
X

0      billionair investor shark tank star mark cuban...
1      attorney jarrett adam recent help overturn inn...
2      the amazon rainforest well known across world ...
3      newark polic : no offic fire singl shot 2020 ,...
4      i 1970 døde det 560 mennesk den norsk trafikke...
                             ...                        
843    frighten detail emerg toilet block attack two ...
844    interview johannesburg — 360,000 adolesc proje...
845    a violent attack southern isra port citi eilat...
846    it 's 17-year-old boy , frighten wear kippa ( ...
847    hong kong ( afp ) - five peopl kill 32 injur c...
Name: text, Length: 848, dtype: object

In [ ]:
def preprocessing(x):
  return re.sub(r"\s+"," ",re.sub(r"\W"," ",x))

In [ ]:
X = X.apply(preprocessing)
X

0      billionair investor shark tank star mark cuban...
1      attorney jarrett adam recent help overturn inn...
2      the amazon rainforest well known across world ...
3      newark polic no offic fire singl shot 2020 tha...
4      i 1970 døde det 560 mennesk den norsk trafikke...
                             ...                        
843    frighten detail emerg toilet block attack two ...
844    interview johannesburg 360 000 adolesc project...
845    a violent attack southern isra port citi eilat...
846    it s 17 year old boy frighten wear kippa relig...
847    hong kong afp five peopl kill 32 injur coach c...
Name: text, Length: 848, dtype: object

In [ ]:
word_count = {}
for words in X:
  for word in words.split():
    if word not in word_count:
      word_count[word]=1
    else:
      word_count[word]=word_count[word]+1


In [ ]:
freq_words = [key for key, value in word_count.items() if value>1]

In [ ]:
def freq_words_filter(x):
  return " ".join([i for i in x.split() if i in freq_words])

In [ ]:
X = X.apply(freq_words_filter)
X

0      billionair investor shark tank star mark cuban...
1      attorney jarrett adam recent help overturn inn...
2      the amazon rainforest well known across world ...
3      newark polic no offic fire singl shot 2020 tha...
4      i 1970 det 560 den norsk trafikken og direktør...
                             ...                        
843    frighten detail emerg toilet block attack two ...
844    interview johannesburg 360 000 adolesc project...
845    a violent attack southern isra port citi eilat...
846    it s 17 year old boy frighten wear religi stre...
847    hong kong afp five peopl kill 32 injur coach c...
Name: text, Length: 848, dtype: object

In [ ]:
from re import T
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
X

0      billionair investor shark tank star mark cuban...
1      attorney jarrett adam recent help overturn inn...
2      the amazon rainforest well known across world ...
3      newark polic no offic fire singl shot 2020 tha...
4      i 1970 det 560 den norsk trafikken og direktør...
                             ...                        
843    frighten detail emerg toilet block attack two ...
844    interview johannesburg 360 000 adolesc project...
845    a violent attack southern isra port citi eilat...
846    it s 17 year old boy frighten wear religi stre...
847    hong kong afp five peopl kill 32 injur coach c...
Name: text, Length: 848, dtype: object

In [ ]:
X = tfidf_vectorizer.fit_transform(X.values)
X = X.toarray()

In [ ]:
len([i for i in X[10] if i>0])

171

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(sampling_strategy ='auto',random_state=0)
# X_train,y_train = smote.fit_resample(X_train,y_train)

# smote_test = SMOTE(sampling_strategy ='auto',random_state=0)
# X_test,y_test = smote_test.fit_resample(X_test,y_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(len(X[0]), activation='relu'))
model.add(Dense(8000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8000, activation='relu'))




model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy')

In [ ]:
X.shape

(848, 12774)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping =EarlyStopping(monitor = 'val_loss', patience = 2)

In [ ]:
model.fit(x=X_train,y=y_train,epochs=10,validation_data = (X_test,y_test), callbacks = [early_stopping])

Epoch 1/10
22/22 [==============================] - 202s 9s/step - loss: 0.4439 - val_loss: 0.2758
Epoch 2/10
22/22 [==============================] - 157s 7s/step - loss: 0.1189 - val_loss: 0.2960
Epoch 3/10
22/22 [==============================] - 139s 6s/step - loss: 0.0330 - val_loss: 0.7978


In [ ]:
pred=model.predict(X_test)

6/6 [==============================] - 5s 804ms/step


In [ ]:
pred = pred.round()

In [ ]:
y_test

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

[[  5  16]
 [  0 149]]
              precision    recall  f1-score   support

         0.0       1.00      0.24      0.38        21
         1.0       0.90      1.00      0.95       149

    accuracy                           0.91       170
   macro avg       0.95      0.62      0.67       170
weighted avg       0.92      0.91      0.88       170



In [ ]:
tfidf_vectorizer.transform([input]).toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

1/1 [==============================] - 0s 135ms/step


array([[1.]], dtype=float32)

In [ ]:
joblib.dump(model,'/content/drive/MyDrive/Colab Notebooks/Projects/News/model.h5')

NameError: name 'joblib' is not defined